In [1]:
import flwr as fl
import tensorflow as tf
import math
import os
# Tensorflow 로그 출력 줄이기
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
# 동시 9명 연합학습을 위해 cpu 사용
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import DataLoader
from collections import OrderedDict

2023-08-10 20:55:45.707413: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-10 20:55:45.805024: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-10 20:55:45.829641: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-08-10 20:55:46.202309: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

## TensorFlow 연합학습 클라이언트 코드

In [2]:
# Tensorflow 로그 출력 줄이기
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

model_fl = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation='softmax')
])

model_fl.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# 하나의 리스트를 n개로 분할하는 함수 정의
def list_split(arr, n):
    num = math.ceil(len(arr) / n)
    return [arr[i: i + num] for i in range(0, len(arr), num)]

x_train_list, y_train_list, x_val_list, y_val_list = map(list_split, (x_train, y_train, x_test, y_test), (3, 3, 3, 3))

class FlowerClient(fl.client.NumPyClient):
    def __init__(self, model, x_train, y_train, x_val, y_val):
        self.model = model
        self.x_train, self.y_train = x_train, y_train
        self.x_val, self.y_val = x_val, y_val

    def get_parameters(self, config):
        return self.model.get_weights() # 모델의 파라미터 반환

    def fit(self, parameters, config):
        self.model.set_weights(parameters) # 서버에서 받은 parameters 모델 적용
        self.model.fit(self.x_train, self.y_train, batch_size=64, epochs=1, verbose=2) # 모델 학습
        return self.model.get_weights(), len(self.x_train), {} # 필수 반환

    def evaluate(self, parameters, config):
        self.model.set_weights(parameters) # 서버에서 받은 parameters 모델 적용
        loss, acc = self.model.evaluate(self.x_val, self.y_val, batch_size=64, verbose=2)
        return loss, len(self.x_val), {"accuracy": acc} # 필수 반환

client_num = 2
flwr_client = FlowerClient(model_fl, x_train_list[client_num], y_train_list[client_num], x_val_list[client_num], y_val_list[client_num])
fl.client.start_numpy_client(server_address="127.0.0.1:8080", client=flwr_client)

2023-08-10 20:55:47.616719: E tensorflow/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-08-10 20:55:47.616799: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: dilab248
2023-08-10 20:55:47.616813: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: dilab248
2023-08-10 20:55:47.616963: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: NOT_FOUND: was unable to find libcuda.so DSO loaded into this program
2023-08-10 20:55:47.617022: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 470.199.2
2023-08-10 20:55:47.618589: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To ena

313/313 - 1s - loss: 0.5211 - accuracy: 0.8491 - 1s/epoch - 4ms/step
53/53 - 0s - loss: 0.1722 - accuracy: 0.9550 - 325ms/epoch - 6ms/step
313/313 - 1s - loss: 0.2619 - accuracy: 0.9238 - 1s/epoch - 3ms/step
53/53 - 0s - loss: 0.1174 - accuracy: 0.9661 - 119ms/epoch - 2ms/step
313/313 - 1s - loss: 0.2048 - accuracy: 0.9419 - 1s/epoch - 3ms/step
53/53 - 0s - loss: 0.0911 - accuracy: 0.9724 - 114ms/epoch - 2ms/step
313/313 - 1s - loss: 0.1742 - accuracy: 0.9503 - 1s/epoch - 3ms/step
53/53 - 0s - loss: 0.0786 - accuracy: 0.9778 - 115ms/epoch - 2ms/step
313/313 - 1s - loss: 0.1536 - accuracy: 0.9552 - 1s/epoch - 3ms/step
53/53 - 0s - loss: 0.0689 - accuracy: 0.9793 - 104ms/epoch - 2ms/step
313/313 - 1s - loss: 0.1380 - accuracy: 0.9591 - 1s/epoch - 3ms/step
53/53 - 0s - loss: 0.0613 - accuracy: 0.9820 - 108ms/epoch - 2ms/step
313/313 - 1s - loss: 0.1251 - accuracy: 0.9637 - 1s/epoch - 3ms/step
53/53 - 0s - loss: 0.0567 - accuracy: 0.9829 - 112ms/epoch - 2ms/step
313/313 - 1s - loss: 0.1160

DEBUG flwr 2023-08-10 20:56:02,237 | connection.py:113 | gRPC channel closed
INFO flwr 2023-08-10 20:56:02,239 | app.py:185 | Disconnect and shut down
